# Standardized data seems to perform slightly better

In [1]:
pd.options.display.max_columns=999

In [2]:
from sklearn.metrics import mean_squared_error
import data_science.lendingclub.dataprep_and_modeling.modeling_utils.data_prep_new as data_prep
import dir_constants as dc
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib
from sklearn.model_selection import RandomizedSearchCV
import time
from tqdm import tqdm_notebook

# DO NOT FORGET TO DROP ISSUE_D AFTER PREPPING

In [3]:
platform = 'lendingclub'
regr_version = '0.2.2'
type_model = 'all_train'
store = pd.HDFStore(f'{datapath}{platform}/{platform}.h5')

In [4]:
if type_model == 'ctest':
    loan_info = store['ctrain']
    eval_cols = store['ctrain_roi']
    std_dev = loan_info.std()
    to_drop = std_dev[std_dev == 0].index.values
    loan_info.drop(to_drop, axis=1, inplace=True)
elif type_model == 'all_train':
    loan_info = store['all_train']
    eval_cols = store['all_train_roi']    
store.close()

# Until I figure out a good imputation method (e.g. bayes PCA), just drop columns with null still

In [5]:
mean_series = loan_info.mean()
std_dev_series = loan_info.std()
standardized = (loan_info-mean_series)/std_dev_series
# standardized = loan_info

# random search RF hyperparams, and build with best one

In [6]:
regr = RandomForestRegressor()

In [7]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [8]:
fit_params_dict = {'n_estimators': np.arange(1,201),
                   'criterion': ['mse'],#, 'mae'
                   'max_features': [3, 50, 70, 100, 150, 200],
                   'min_samples_split': [20, 200, 2000],
                   'min_samples_leaf': [10, 100, 1000],
                   'bootstrap': [True],
                   'oob_score': [True],
                   'n_jobs': [-1],
                   'verbose': [10]}

In [9]:
n_iter = 20
random_search = RandomizedSearchCV(
    regr, param_distributions=fit_params_dict, n_iter=n_iter)

In [10]:
start = time.time()
random_search.fit(standardized, eval_cols)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time.time() - start), n_iter))

/home/justin/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


building tree 1 of 85building tree 5 of 85building tree 2 of 85building tree 4 of 85building tree 6 of 85building tree 3 of 85building tree 7 of 85building tree 8 of 85







building tree 9 of 85


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   11.2s


building tree 10 of 85
building tree 11 of 85
building tree 12 of 85
building tree 13 of 85
building tree 14 of 85
building tree 15 of 85
building tree 16 of 85


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   21.8s


...

building tree 121 of 154
building tree 122 of 154
building tree 123 of 154
building tree 124 of 154
building tree 125 of 154
building tree 126 of 154
building tree 127 of 154
building tree 128 of 154
building tree 129 of 154
building tree 130 of 154
building tree 131 of 154
building tree 132 of 154
building tree 133 of 154
building tree 134 of 154
building tree 135 of 154
building tree 136 of 154


[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  4.6min


building tree 137 of 154
building tree 138 of 154
building tree 139 of 154
building tree 140 of 154
building tree 141 of 154
building tree 142 of 154
building tree 143 of 154
building tree 144 of 154
building tree 145 of 154
building tree 146 of 154
building tree 147 of 154
building tree 148 of 154
building tree 149 of 154
building tree 150 of 154
building tree 151 of 154
building tree 152 of 154
building tree 153 of 154
building tree 154 of 154


[Parallel(n_jobs=-1)]: Done 154 out of 154 | elapsed:  5.4min finished


RandomizedSearchCV took 12168.27 seconds for 20 candidates parameter settings.


In [11]:
report(random_search.cv_results_)
# Model with rank: 1
# Mean validation score: 0.026 (std: 0.010)
# Parameters: {'verbose': 10, 'oob_score': True, 'n_jobs': -1, 'n_estimators': 81, 'min_samples_split': 200, 'min_samples_leaf': 10, 'max_features': 150, 'criterion': 'mse', 'bootstrap': True}

# Model with rank: 2
# Mean validation score: 0.025 (std: 0.014)
# Parameters: {'verbose': 10, 'oob_score': True, 'n_jobs': -1, 'n_estimators': 159, 'min_samples_split': 20, 'min_samples_leaf': 100, 'max_features': 150, 'criterion': 'mse', 'bootstrap': True}

# Model with rank: 3
# Mean validation score: 0.025 (std: 0.014)
# Parameters: {'verbose': 10, 'oob_score': True, 'n_jobs': -1, 'n_estimators': 186, 'min_samples_split': 200, 'min_samples_leaf': 100, 'max_features': 100, 'criterion': 'mse', 'bootstrap': True}

Model with rank: 1
Mean validation score: 0.027 (std: 0.017)
Parameters: {'verbose': 10, 'oob_score': True, 'n_jobs': -1, 'n_estimators': 154, 'min_samples_split': 200, 'min_samples_leaf': 10, 'max_features': 50, 'criterion': 'mse', 'bootstrap': True}

Model with rank: 2
Mean validation score: 0.022 (std: 0.021)
Parameters: {'verbose': 10, 'oob_score': True, 'n_jobs': -1, 'n_estimators': 195, 'min_samples_split': 20, 'min_samples_leaf': 100, 'max_features': 50, 'criterion': 'mse', 'bootstrap': True}

Model with rank: 3
Mean validation score: 0.022 (std: 0.018)
Parameters: {'verbose': 10, 'oob_score': True, 'n_jobs': -1, 'n_estimators': 147, 'min_samples_split': 200, 'min_samples_leaf': 100, 'max_features': 100, 'criterion': 'mse', 'bootstrap': True}



# take best hyperparams from report

In [12]:
regr = RandomForestRegressor(
    n_estimators=154,
    random_state=0,
    max_features=min(standardized.shape[1],50),
    min_samples_split=200,
    min_samples_leaf=10,
    n_jobs=-1,
    oob_score=True,
    bootstrap=True,
    criterion='mse',
)
regr.fit(standardized, eval_cols)
# dump the model
joblib.dump(regr, f'{datapath}{platform}/model_dump/standardized_{regr_version}_{type_model}.pkl')
joblib.dump((mean_series, std_dev_series), f'{datapath}{platform}/model_dump/mean_stddev_{regr_version}_{type_model}.pkl')

/home/justin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':


['/home/justin/all_data/lendingclub/model_dump/mean_stddev_0.2.2_all_train.pkl']

In [13]:
regr.score(standardized, eval_cols)

0.18549187491311636

In [14]:
ls /home/justin/all_data/lendingclub/model_dump/

0.2.2_ctest.pkl                  nonstandardized_0.2.2_ctest.pkl
mean_stddev_0.2.2_all_train.pkl  standardized_0.2.2_all_train.pkl
mean_stddev_0.2.2_ctest.pkl      standardized_0.2.2_ctest.pkl


# non standardized regressor

In [10]:
regr = RandomForestRegressor(
    n_estimators=81,
    random_state=0,
    max_features=min(standardized.shape[1],150),
    min_samples_split=200,
    min_samples_leaf=10,
    n_jobs=-1,
    oob_score=True,
    bootstrap=True,
    criterion='mse',
)
regr.fit(loan_info, eval_cols)
# dump the model
joblib.dump(regr, f'{datapath}{platform}/model_dump/nonstandardized_{regr_version}_{type_model}.pkl')
joblib.dump((mean_series, std_dev_series), f'{datapath}{platform}/model_dump/mean_stddev_{regr_version}_{type_model}.pkl')

/home/justin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':


['/home/justin/all_data/lendingclub/model_dump/mean_stddev_0.2.2_ctest.pkl']

In [15]:
regr.score(loan_info, eval_cols)

0.17217201887632605

In [9]:
now = time.strftime("%Y_%m_%d_%Hh_%Mm_%Ss")
# info to stick in detailed dataframe describing each model
model_info = {'model_version': regr_version,
              'target': 'npv_roi_7',
              'weights': 'None',
              'algo_model': 'RF_regr',
              'hyperparams': "bootstrap=True, criterion='mse', max_depth=None, max_features=150, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=10, min_samples_split=200, min_weight_fraction_leaf=0.0, n_estimators=81, n_jobs=-1, oob_score=True, random_state=0, verbose=0, warm_start=False",
              'cost_func': 'sklearn default, which I think is mse',
              'useful_notes': 'R2 score of 0.1667047 (regr.score())',
              'date': now}

model_info_df = pd.DataFrame(model_info, index = [regr_version])

In [10]:
store.open()
store['model_info'] = model_info_df
store.close()

In [11]:
# store.open()
# model_info = store['model_info']
# model_info.ix[regr_version,:] = model_info_df.values
# model_info.sort_index(inplace=True)
# store.append(
#             'model_info',
#             model_info_df,
#             data_columns=True,
#             index=True,
#             append=False,
# )
# store.close()

# Examine performance on test set

In [8]:
store.open()
test = store['ctest']
train = store['ctrain']
if type_model == 'ctest':
    std_dev = train.std()
    to_drop = std_dev[std_dev == 0].index.values
    train.drop(to_drop, axis=1, inplace=True)
    test.drop(to_drop, axis=1, inplace=True)
    
loan_npv_rois = store['loan_npv_rois']
default_series = store['issue_d_id_mats']['target_loose']
results = store['results']
store.close()

In [12]:
regr_version = '0.2.2'
# regr = joblib.load(f'{datapath}{platform}/model_dump/standardized_{regr_version}_{type_model}.pkl')
regr_std = joblib.load(f'{datapath}{platform}/model_dump/standardized_{regr_version}_{type_model}.pkl')
mean_series, std_dev_series = joblib.load(f'{datapath}{platform}/model_dump/mean_stddev_{regr_version}_{type_model}.pkl')

In [10]:
test_std = (test-mean_series)/std_dev_series
train_std = (train-mean_series)/std_dev_series

In [13]:
# test_yhat = regr.predict(test)
# train_yhat = regr.predict(train)
test_std_yhat = regr_std.predict(test_std)
train_std_yhat = regr_std.predict(train_std)

In [14]:
# print(regr.score(test, loan_npv_rois.loc[test.index].values))
# print(regr.score(train, loan_npv_rois.loc[train.index].values))
print(regr_std.score(test_std, loan_npv_rois.loc[test.index].values))
print(regr_std.score(train_std, loan_npv_rois.loc[train.index].values))

0.00974506745554
0.17217376239


In [14]:
# test_mse = mean_squared_error(test_yhat, loan_npv_rois.loc[test.index].values)
# train_mse = mean_squared_error(train_yhat, loan_npv_rois.loc[train.index].values)

In [15]:
test_yhat_series = pd.Series(test_std_yhat, index = test.index)

In [16]:
def eval_models(trials, port_size, available_loans, regr, regr_version, test, loan_npv_rois,
                default_series,test_yhat_series):
    results = {}
    pct_default = {}
    test_copy = test.copy()
    for trial in tqdm_notebook(np.arange(trials)):
        loan_ids = np.random.choice(
            test_copy.index.values, available_loans, replace=False)
        loans_to_pick_from = test_copy.loc[loan_ids, :]
        scores = test_yhat_series.loc[loan_ids].values
        scores_series = pd.Series(dict(zip(loan_ids, scores)))
        scores_series.sort_values(ascending=False, inplace=True)
        picks = scores_series[:port_size].index.values
        results[trial] = loan_npv_rois.loc[picks, :].mean().to_dict()
        pct_default[trial] = (default_series.loc[picks].sum()) / port_size
    pct_default_series = pd.Series(pct_default)
    results_df = pd.DataFrame(results).T
    results_df['pct_def'] = pct_default_series
    return results_df

In [17]:
# as per done with baseline models, say 3000 loans available
# , pick 900 of them
trials = 30000
port_size = 150
available_loans = 500
model_results = eval_models(trials, port_size, available_loans, regr_std, regr_version, test=test_std, loan_npv_rois=loan_npv_rois, default_series=default_series, test_yhat_series=test_yhat_series)

A Jupyter Widget

In [18]:
multi_index = []
for col in model_results.columns.values:
    multi_index.append((str(col),regr_version))
append_results = model_results.copy()
append_results.columns = pd.MultiIndex.from_tuples(multi_index, names = ['discount_rate', 'model'])    

In [19]:
multi_index_results = []
for col in results.columns.values:
    multi_index_results.append((str(col[0]), col[1]))
results.columns = pd.MultiIndex.from_tuples(multi_index_results, names = ['discount_rate', 'model'])    

In [20]:
try:
    results = results.join(append_results)
except ValueError:
    results.loc[:,append_results.columns] = append_results
# results.sort_index(axis=1, inplace = True)

In [23]:
results.describe()

discount_rate,0,pct_def,0,pct_def,0.07,pct_def
model,baseline,baseline,1.00001,1.00001,0.2.2,0.2.2
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000
mean,-0.059090,0.197007,-0.031466,0.069048,-0.027506,0.116026
std,0.018949,0.032410,0.010804,0.020435,0.014404,0.025727
min,-0.148132,0.066667,-0.086961,0.006667,-0.092307,0.026667
25%,-0.071569,0.173333,-0.038453,0.053333,-0.037009,0.100000
50%,-0.058625,0.193333,-0.030865,0.066667,-0.026875,0.113333
75%,-0.046024,0.220000,-0.023836,0.080000,-0.017538,0.133333
max,0.014950,0.333333,0.001349,0.160000,0.023947,0.220000


In [22]:
store.open()
store['results'] = results
store.close()

/home/justin/anaconda3/lib/python3.6/site-packages/pandas/io/pytables.py:486: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->axis0_level1] [items->None]

  self.put(key, value)
/home/justin/anaconda3/lib/python3.6/site-packages/pandas/io/pytables.py:486: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_items_level1] [items->None]

  self.put(key, value)
